In [19]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkFiles
from pyspark.sql.functions import mean
import pyspark.sql.functions as f
from pyspark.ml.stat import Correlation
import csv
import pandas as pd
from pyspark.sql.functions import col

In [2]:
sc = SparkContext('local[16]')
spark = SQLContext(sc)

22/06/17 14:33:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.csv('/data/dataprocessing/interproscan/all_bacilli.tsv', sep=r'\t', header=False)#.select('col1','col2')

In [4]:
df = df.toDF('Protein_accession','Sequence_MD5_digest','Sequence_length','Analysis','Signature_accession',
             'Signature_description','Start_location','Stop_location','Score','Status','Date','INT_accession',
             'INT_description','GO_annotations','Pathways_annotations')

In [5]:
# yoink
def explain_func(data):
    return data._sc._jvm.PythonSQLUtils.explainString(data._jdf.queryExecution(), 'simple')

1. How many distinct protein annotations are found in the dataset? I.e. how many distinc InterPRO numbers are there?
2. How many annotations does a protein have on average?
3. What is the most common GO Term found?
4. What is the average size of an InterPRO feature found in the dataset?
5. What is the top 10 most common InterPRO features?
6. If you select InterPRO features that are almost the same size (within 90-100%) as the protein itself, what is the top10 then?
7. If you look at those features which also have textual annotation, what is the top 10 most common word found in that annotation?
8. And the top 10 least common?
9. Combining your answers for Q6 and Q7, what are the 10 most commons words found for the largest InterPRO features?
10. What is the coefficient of correlation ($R^2$) between the size of the protein and the number of features found?

In [6]:
#1
quest1 = df.select('INT_accession').distinct().count()
quest1_explain = explain_func(df.select('INT_accession').distinct())

In [7]:
#2. 
quest2 = df.select('Protein_accession').count() / df.select('Protein_accession').distinct().count()
quest2_explain = explain_func(df.select('Protein_accession')), explain_func(df.select('Protein_accession'))

In [9]:
#3
quest3df = df[df.GO_annotations.contains('GO')]
quest3 = quest3df.groupby('GO_annotations').count().orderBy("count", ascending=False).first()[0]
quest3_explain = explain_func(quest3df.groupby('GO_annotations').count().orderBy("count", ascending=False))

In [10]:
#4
# Its either this:
quest4df = df.withColumn('lenght', ( df['Stop_location'] - df['Start_location'] ))
quest4 = quest4df.select(mean('lenght')).collect()[0][0]
quest4_explain = explain_func(quest4df.select(mean('lenght')))
# Or this:
# quest4 = df.select(mean ('Sequence_length')).collect()[0][0]

In [11]:
#5
quest5df = df[df.INT_accession.contains('IPR0')]
quest5_list = quest5df.groupby('INT_accession').count().orderBy("count", ascending=False).take(10)
quest5_explain = explain_func(quest5df.groupby('INT_accession').count().orderBy("count", ascending=False))
quest5 = []
for nr in range(0,len(quest5_list)):
    quest5.append(quest5_list[nr][0])

In [12]:
#6
quest6df = quest5df.withColumn('lenght', ( df['Stop_location'] - df['Start_location'] ) / df['Sequence_length'])
quest6df2 = quest6df[quest6df['lenght'] > 0.9]
quest6_list = quest6df2.groupby('INT_accession').count().orderBy("count", ascending=False).take(10)
quest6_explain = explain_func(quest6df2.groupby('INT_accession').count().orderBy("count", ascending=False))
quest6 = []
for nr in range(0,len(quest6_list)):
    quest6.append(quest6_list[nr][0])

In [13]:
#7
# Thank you stackoverflow https://stackoverflow.com/questions/48927271/count-number-of-words-in-a-spark-dataframe 
quest7df = df[df.INT_accession.isin('-') == False]
quest7_list = quest7df.withColumn('word', f.explode(f.split(f.col('INT_description'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .take(10)
quest7_explain = explain_func(quest7df.withColumn('word', f.explode(f.split(f.col('INT_description'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False))
quest7 = []
for nr in range(0,len(quest7_list)):
    quest7.append(quest7_list[nr][0])

In [14]:
#8
quest8_list = quest7df.withColumn('word', f.explode(f.split(f.col('INT_description'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=True)\
    .take(10)
quest8_explain = explain_func(quest7df.withColumn('word', f.explode(f.split(f.col('INT_description'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=True))
quest8 = []
for nr in range(0,len(quest8_list)):
    quest8.append(quest8_list[nr][0])

In [15]:
#9
quest9_list = quest6df2.withColumn('word', f.explode(f.split(f.col('INT_description'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .take(10)
quest9_explain = explain_func(quest6df2.withColumn('word', f.explode(f.split(f.col('INT_description'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False))
quest9 = []
for nr in range(0,len(quest9_list)):
    quest9.append(quest9_list[nr][0])

In [24]:
quest10df = quest6df
quest10df = quest10df.withColumn('Sequence_length', quest10df['Sequence_length'].cast("int"))

In [25]:
#10
quest10df = quest6df
quest10df = quest10df.withColumn('Sequence_length', quest10df['Sequence_length'].cast("int"))
quest10df2 = quest7df.groupby('INT_accession').count()
quest10df = quest10df.join(quest10df2,'INT_accession')
quest10_explain = explain_func(quest10df)
quest10 = quest10df.corr('Sequence_length', 'count')

In [27]:
# Making the csv
output = {'question_nr' : range(1,11),
 'answers' : [quest1, quest2, quest3, quest4, quest5, quest6, quest7, quest8, quest9, quest10],
 'explain' : [quest1_explain, quest2_explain, quest3_explain, quest4_explain, quest5_explain, quest6_explain, quest7_explain, quest8_explain, quest9_explain, quest10_explain],
}
output = pd.DataFrame(output)
output.to_csv('/homes/rdalstra/Documents/Programming3/Assignment5/output/answers.csv', sep=',', index=False)